In [1]:
# Imports
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm

In [2]:
MODEL_DIR = "cifar_cnn_model"
BATCH_SIZE = 200

# Data Loading Definition

In [3]:
# Imports
import numpy as np

from tqdm import tqdm

import cv2
import glob
import sys

import os

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return cv2.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split(os.sep)
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)
    

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    files = get_files(folder)
    images = []
    tqdm_iter = tqdm(files)
    tqdm_iter.set_description("Image Data Loaded -> ")
    for index, f in enumerate(tqdm_iter):
        img_arr = get_img_array(f)
        images.append(img_arr)
    X = np.array(images)
    return X

def get_labels(folder, label2id):
    files = get_files(folder)
    y = []
    tqdm_iter = tqdm(files)
    tqdm_iter.set_description("Label Data Loaded -> ")
    for index, f in enumerate(tqdm_iter):
        y.append(get_label(f,label2id))
    y = np.array(y)

    return y

def get_images_and_labels(folder, label2id):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """ 

    return get_images(folder), get_labels(folder,label2id)

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = os.path.join(data_root_path, 'train')
    id2label, label2id = get_label_mapping(os.path.join(data_root_path,'labels.txt'))
    print(label2id)
    print('Loading training data')
    X, y = get_images_and_labels(train_data_path,label2id)
    print('Training data loaded.\n')
    return X, y

def train_val_split(X, y, training_rate=0.8):
    n = X.shape[0]
    num = int(n * training_rate)
    print('Split training/validation data->training:{}, validation:{}'.format(num, n - num))
    print('Splitting done')
    return X[0:num], y[0:num], X[num:], y[num:]

# Model Definition

In [4]:
import numpy as np
import tensorflow as tf

# tf.logging.set_verbosity(tf.logging.INFO)

def inference(features,is_training=True,dropout_rate=0.4,num_classes=10):
  
  def data_aug(raw_images):
    distorted_images = tf.map_fn(lambda image:tf.image.random_flip_left_right(image),raw_images)
    distorted_images = tf.map_fn(lambda image:tf.image.random_flip_up_down(image),distorted_images)

    # Because these operations are not commutative, consider randomizing
    # the order their operation.
    # NOTE: since per_image_standardization zeros the mean and makes
    # the stddev unit, this likely has no effect see tensorflow#1458.
    distorted_images = tf.image.random_brightness(distorted_images,
                                             max_delta=63)
    distorted_images = tf.image.random_contrast(distorted_images,
                                           lower=0.2, upper=1.8)
    return distorted_images
    
  def norm_input(raw_images):
    return tf.map_fn(lambda distorted_image:tf.image.per_image_standardization(distorted_image),raw_images)

  images = features['x']
  with tf.device('/cpu:0'):
      # Randomly flip the image horizontally.
      if is_training:
          images = data_aug(images)
      # Subtract off the mean and divide by the variance of the pixels.
      images = norm_input(images)
  
  # Input Layer
  input_layer = tf.reshape(images, [-1, 32, 32, 3])

  # Convolutional Layer #1
  # Input:  [-1,32,32,3]
  # Output: [-1,32,32,32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu,
      kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=0.01))

  # Pooling Layer #1
  # Input: [-1,32,32,32]
  # Output: [-1,16,16,32]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    
  # Norm #1
  norm1 = tf.layers.batch_normalization(inputs=pool1)

  # Convolutional Layer #2, Norm#2 and Pooling Layer #2
  # Input: [-1,16,16,32]
  # Output: [-1,16,16,64]
  conv2 = tf.layers.conv2d(
      inputs=norm1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu,
      kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=0.01))
  

  # norm2
  norm2 = tf.layers.batch_normalization(inputs=conv2)
  # Input: [-1,16,16,64]
  # Output: [-1,8,8,64]
  pool2 = tf.layers.max_pooling2d(inputs=norm2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 8 * 8 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu, kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=0.01))
  dropout = tf.layers.dropout(
      inputs=dense, rate=dropout_rate, training=is_training)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=num_classes)
  return logits


def get_train_op_fn(loss, lr=0.01):
    """Get the training Op.
    Args:
         loss (Tensor): Scalar Tensor that represents the loss function.
    Returns:
        Training Op
    """
    return tf.contrib.layers.optimize_loss(
        loss=loss,
        global_step=tf.train.get_global_step(),
        optimizer=tf.train.AdamOptimizer,
        learning_rate=lr
    )

def calculate_loss(logits,labels):
    training_loss = tf.losses.sparse_softmax_cross_entropy(labels=labels,logits=logits)
    l2_loss = tf.losses.get_regularization_loss()
    return training_loss+l2_loss

def get_eval_metric_ops(labels, predictions):
    accuracy = tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])
    return {'accuracy':accuracy}

# Our application logic will be added here
def cnn_model_fn(features, labels, mode):
  """Model function used in the estimator.
    Args:
      features (Tensor): Input features to the model.
      labels (Tensor): Labels tensor for training and evaluation.
      mode (ModeKeys): Specifies if training, evaluation or prediction.
    Returns:
      (EstimatorSpec): Model to be run by Estimator.
  """

  # inference
  logits = inference(features, is_training=mode == tf.estimator.ModeKeys.TRAIN, num_classes=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # TODO
  # Decay the learning rate exponentially based on the number of steps.
                 
  # Calculate arguments (for both TRAIN and EVAL modes)
  loss = calculate_loss(logits,labels)
  eval_metric_ops = get_eval_metric_ops(labels, predictions)
  tf.summary.scalar("acc", eval_metric_ops['accuracy'][1])
  summary_hook = tf.train.SummarySaverHook(100,output_dir=os.path.join(MODEL_DIR,'train'), summary_op=tf.summary.merge_all())

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:             
    train_op = get_train_op_fn(loss, 0.01)
    
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op, training_hooks=[summary_hook],eval_metric_ops=eval_metric_ops)
  
  # Add evaluation metrics (for EVAL mode)
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops,evaluation_hooks=[summary_hook])

# Experiment Definition

In [5]:
import tensorflow as tf

# Show debugging output
tf.logging.set_verbosity(tf.logging.DEBUG)

# Define model ############################################
def get_estimator(model_fn,model_dir):
    """Return the model as a Tensorflow Estimator object.
    Args:
         run_config (RunConfig): Configuration for Estimator run.
    """
    return tf.estimator.Estimator(
        model_fn=model_fn,
        model_dir=model_dir
    )

# Load data

In [6]:
DATA_ROOT_PATH = os.path.join('data','HW2_data')

In [7]:
X, y = get_train_data(DATA_ROOT_PATH)
X = np.asarray(X,dtype=np.float32)
X_train, y_train, X_val, y_val = train_val_split(X, y)

{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
Loading training data


Label Data Loaded -> : 100%|█████████████████████████████████████████████████| 45000/45000 [00:00<00:00, 786913.98it/s]


Training data loaded.

Split training/validation data->training:36000, validation:9000
Splitting done


# Model Creation

In [8]:
cifar_classifier = get_estimator(model_fn=cnn_model_fn,model_dir=MODEL_DIR)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'cifar_cnn_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000233DD0A30B8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# Train Model

In [10]:
TRAIN_STEP = 1000#X_train.shape[0]
TRAIN_EPOCH = int(TRAIN_STEP / BATCH_SIZE)
print('Total Training Epochs:', TRAIN_EPOCH)

Total Training Epochs: 5


In [11]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=BATCH_SIZE,
    num_epochs=None,
    shuffle=True)
cifar_classifier.train(
  input_fn=train_input_fn,
  steps=TRAIN_STEP)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from cifar_cnn_model\model.ckpt-1
INFO:tensorflow:Saving checkpoints for 2 into cifar_cnn_model\model.ckpt.
INFO:tensorflow:loss = 93.54839, step = 2
INFO:tensorflow:global_step/sec: 5.5163
INFO:tensorflow:loss = 3.9210865, step = 102 (18.130 sec)


KeyboardInterrupt: 

# Evaluation Model

## Training Dataset Evaluation

In [ ]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    num_epochs=1,
    shuffle=False)
eval_results = cifar_classifier.evaluate(input_fn=eval_input_fn)
print('Training evaluation:')
print(eval_results)

## Validation Dataset Evaluation

In [ ]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_val},
    y=y_val,
    num_epochs=1,
    shuffle=False)
eval_results = cifar_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

# Prediction

In [ ]:
TEST_PATH = os.path.join(DATA_ROOT_PATH, 'test')
ID2LABEL, _ = get_label_mapping(os.path.join(DATA_ROOT_PATH,'labels.txt'))
MAX_IMG = 10
IMG_ROW = 2
IMG_COL = MAX_IMG / IMG_ROW
OUTPUT_FILE = 'submit.csv'

## Load Test Data

In [ ]:
X_test = get_images(TEST_PATH).astype(dtype=np.float32)
filenames = get_files(TEST_PATH)

## Predict Result

In [ ]:
# set prediction tensors
pred_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": X_test},y=None, num_epochs=1, shuffle=False)
predictions = cifar_classifier.predict(input_fn=pred_input_fn)

# sample plot
fig=plt.figure(figsize=(8, 8))

# dataframe to store and save results
df = pd.DataFrame(columns=['id','label'])

# prediction progress bar
tqdm_iter = tqdm(list(zip(predictions,filenames)))
tqdm_iter.set_description('Predicted -> ')

# image count
img_count = 0

# iterations
for index, (prediction, filename) in enumerate(tqdm_iter):
    # extract picture id
    pic_id = int(filename.split(os.sep)[-1].split('.')[0])
    
    # extract label
    label = ID2LABEL[prediction['classes']]
    
    # store information
    df = df.append({'id':pic_id - 1,'label':label},ignore_index=True,verify_integrity=True)
    
    # plot first MAX_IMG images
    if img_count < MAX_IMG and np.random.random() >= 0.9:
        img_count += 1
        img=mpimg.imread(filename)
        fig.add_subplot(IMG_ROW, IMG_COL, img_count)
        plt.title(label)
        plt.imshow(img)

# sort by picture id
df = df.sort_values(by=['id']).set_index(['id'])

# save values into csv file
df.to_csv(OUTPUT_FILE)

In [ ]:
# show
df